## Modules & Settings

In [1]:
import os
import sys
sys.path.append('..')

from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bsoption.nyopchain import NYopchain
Opchain = NYopchain()

## Option chain query examples

### Example 1: single expiry, two adjacent trading days, only put, order by strike then tradedate

In [3]:
inputdict1 = {'asset': 'NVDA',  
              'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 150,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 180,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': '2022-08-27',  # query expiry startdate (inclusive)
              'endexpiry': '2022-09-02',  # query expiry enddate (inclusive)              
              'starttd': '2022-08-25',  # query period startdate (inclusive)
              'endtd': '2022-08-26'  # query period startdate (inclusive) 
             }

orderfield1 = ['optype', 'strike', 'tradedate']
dfop1 = Opchain.loadopdata(inputdict1, orderfield=orderfield1)
dfop1

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-08-25,NVDA,P,2022-09-02,150.0,62.89,3692.0,3178,0.17,0.15,0.18,0.165
1,2022-08-26,NVDA,P,2022-09-02,150.0,57.96,10752.0,3246,1.05,1.02,1.05,1.035
2,2022-08-25,NVDA,P,2022-09-02,152.5,61.13,685.0,1050,0.23,0.21,0.24,0.225
3,2022-08-26,NVDA,P,2022-09-02,152.5,57.30,3013.0,1337,1.48,1.47,1.49,1.480
4,2022-08-25,NVDA,P,2022-09-02,155.0,59.38,1897.0,1831,0.30,0.29,0.32,0.305
5,2022-08-26,NVDA,P,2022-09-02,155.0,56.57,5881.0,1954,2.07,2.05,2.07,2.060
6,2022-08-25,NVDA,P,2022-09-02,157.5,57.72,1360.0,1108,0.42,0.40,0.43,0.415
7,2022-08-26,NVDA,P,2022-09-02,157.5,55.71,4499.0,1299,2.77,2.77,2.81,2.790
8,2022-08-25,NVDA,P,2022-09-02,160.0,56.06,8675.0,3841,0.56,0.55,0.57,0.560
9,2022-08-26,NVDA,P,2022-09-02,160.0,54.88,26134.0,6545,3.73,3.65,3.75,3.700


### Example 2:  Single option contract across days

In [4]:
inputdict2 = {'asset': 'QQQ',  
              'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 270,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 270,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': '2022-10-14',  # query expiry startdate (inclusive)
              'endexpiry': '2022-10-14',  # query expiry enddate (inclusive)              
              'starttd': '2022-10-01',  # query period startdate (inclusive)
              'endtd': '2022-10-14'  # query period startdate (inclusive) 
             }

dfop2 = Opchain.loadopdata(inputdict2)
dfop2

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-10-03,QQQ,P,2022-10-14,270.0,35.38,34892.0,33224,5.01,4.94,5.04,4.990
1,2022-10-04,QQQ,P,2022-10-14,270.0,37.33,12617.0,4246,2.38,2.36,2.39,2.375
2,2022-10-05,QQQ,P,2022-10-14,270.0,36.52,13483.0,11539,2.05,2.03,2.05,2.040
3,2022-10-06,QQQ,P,2022-10-14,270.0,38.92,10090.0,18672,2.60,2.56,2.60,2.580
4,2022-10-07,QQQ,P,2022-10-14,270.0,37.27,21365.0,18806,5.94,5.85,6.01,5.930
5,2022-10-10,QQQ,P,2022-10-14,270.0,43.73,6406.0,14919,6.72,6.76,6.90,6.830
6,2022-10-11,QQQ,P,2022-10-14,270.0,45.51,5813.0,13802,8.73,8.64,8.93,8.785
7,2022-10-12,QQQ,P,2022-10-14,270.0,54.03,2445.0,0,8.79,8.81,8.97,8.890
8,2022-10-13,QQQ,P,2022-10-14,270.0,51.05,18217.0,11737,3.36,3.20,3.50,3.350


### Last trading day's option chain with strike in a price range

In [5]:
lasttd = datetime.today() - timedelta(days=1, hours=6)
ltdstr = lasttd.strftime('%Y-%m-%d')
nextexpiry = (lasttd + timedelta(days=7)).strftime('%Y-%m-%d')
next2expiry = (lasttd + timedelta(days=13)).strftime('%Y-%m-%d')

inputdict3 = {'asset': 'AAPL',  
              'optype': ('C', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 140,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 160,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': nextexpiry,  # query expiry startdate (inclusive)
              'endexpiry': next2expiry,  # query expiry enddate (inclusive)              
              'starttd': ltdstr,  # query period startdate (inclusive)
              'endtd': ltdstr  # query period startdate (inclusive) 
             }

dfop3 = Opchain.loadopdata(inputdict3)
dfop3

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-11-04,AAPL,C,2022-11-11,140.0,41.41,42752.0,3474.0,2.39,2.36,2.44,2.400
1,2022-11-04,AAPL,C,2022-11-11,141.0,41.21,13305.0,2492.0,2.04,1.93,2.04,1.985
2,2022-11-04,AAPL,C,2022-11-11,142.0,40.45,21263.0,2475.0,1.60,1.58,1.65,1.615
3,2022-11-04,AAPL,C,2022-11-11,143.0,40.31,14433.0,2467.0,1.32,1.25,1.35,1.300
4,2022-11-04,AAPL,C,2022-11-11,144.0,39.45,12271.0,1942.0,1.04,1.00,1.05,1.025
5,2022-11-04,AAPL,C,2022-11-11,145.0,39.21,27585.0,5117.0,0.82,0.78,0.83,0.805
6,2022-11-04,AAPL,C,2022-11-11,146.0,38.82,7329.0,3085.0,0.62,0.61,0.64,0.625
7,2022-11-04,AAPL,C,2022-11-11,147.0,39.01,5884.0,2087.0,0.48,0.48,0.51,0.495
8,2022-11-04,AAPL,C,2022-11-11,148.0,38.57,4585.0,2425.0,0.37,0.35,0.38,0.365
9,2022-11-04,AAPL,C,2022-11-11,149.0,38.87,9168.0,3957.0,0.27,0.23,0.30,0.265
